In [1]:
import numpy as np
import os
import glob
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import patches 

In [2]:
def get_rectangle_edges_from_pascal_bbox(bbox):
    xmin_top_left, ymin_top_left, xmax_bottom_right, ymax_bottom_right = bbox

    bottom_left = (xmin_top_left, ymax_bottom_right)
    width = xmax_bottom_right - xmin_top_left
    height = ymin_top_left - ymax_bottom_right

    return bottom_left, width, height

def draw_pascal_voc_bboxes(
    plot_ax,
    bboxes,
    get_rectangle_corners_fn=get_rectangle_edges_from_pascal_bbox,
):
    for bbox in bboxes:
        bottom_left, width, height = get_rectangle_corners_fn(bbox)

        rect_1 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=4,
            edgecolor="black",
            fill=False,
        )
        rect_2 = patches.Rectangle(
            bottom_left,
            width,
            height,
            linewidth=2,
            edgecolor="white",
            fill=False,
        )

        # Add the patch to the Axes
        plot_ax.add_patch(rect_1)
        plot_ax.add_patch(rect_2)

def show_image(
    image, bboxes=None, draw_bboxes_fn=draw_pascal_voc_bboxes, figsize=(10, 10)
):
    fig, ax = plt.subplots(1, figsize=figsize)
    ax.imshow(image)

    if bboxes is not None:
        draw_bboxes_fn(ax, bboxes)

    plt.show()

In [3]:
def show_boxes(img, annotations):
    # Convert to Pascal VOC format
    pascal_bboxes = []
    for ann in annotations:
        ann = ann.replace('\n','').split(" ")
        ann = [float(a) for a in ann]
        xcoord = ann[1]*512
        ycoord = ann[2]*512
        width = ann[3]*512
        height = ann[4]*512
        xmin = xcoord - int(width/2)
        xmax = xcoord + int(width/2)
        ymin = ycoord - int(height/2)
        ymax = ycoord + int(height/2)
        pascal_bboxes.append([xmin, ymin, xmax, ymax])
    pascal_bboxes = np.array(pascal_bboxes)
   
    class_labels = np.array([ann[0] for ann in annotations])
    show_image(img, pascal_bboxes.tolist())

def delete_img_small(annotations):
    delete = []
    for ann in annotations:
        ann = ann.replace('\n','').split(" ")
        ann = [float(a) for a in ann]
        xcoord = ann[1]*512
        ycoord = ann[2]*512
        width = ann[3]*512
        height = ann[4]*512
        if (width*height) < 6000:
            delete.append(True)
        else:
            delete.append(False)
    if all(delete):
        return True
    else:
        return False

def find_zero_coords(annotations):
    annotations = [ann.replace('\n','').split(" ") for ann in annotations]
    for ann in annotations:
        ann = [float(a) for a in ann]
        if any([a <= 0 for a in ann[1:]]):
            return True
        else:
            return False


In [17]:
f1 = "C:/fyp/hematology/Data/WBC_20220215/full_image/Purple/tiled2/*.png"
f2 = "T:/FYP_dataset/mixed/"
img_files = sorted(glob.glob(f2 + "*.png"))
label_files = sorted(glob.glob(f2 + "*.txt"))
for imgf,label_file in list(zip(img_files, label_files)):
    img = Image.open(imgf)
    with open(label_file, "r") as f:
        annotations = f.readlines()
        delete = find_zero_coords(annotations)
        #show_boxes(img, annotations)
    #     delete = delete_img_small(annotations)
    if delete:
        os.remove(imgf)
        os.remove(label_file)